In [1]:
import pandas as pd
import lightgbm as lgb
from darts import TimeSeries
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
import numpy as np


In [2]:
data = pd.read_csv('../raw_data/hourly_final_dataset.csv')
df = data


In [3]:
feature_cols = df[["temperature_2m (°C)", "relative_humidity_2m (%)", "apparent_temperature (°C)", "precipitation (mm)", "rain (mm)", "wind_speed_10m (km/h)", "wind_speed_100m (km/h)", "day_of_week_sin", "day_of_week_cos", "month_sin", "month_cos", "isHoliday"]]
past_cov = df[["traditional_baguette", "croissant", "coupe", "pain_au_chocolat", "baguette", "banette", "cereal_baguette"]]


In [4]:
y_train = past_cov.loc[:'2021-12-31']
y_val = past_cov.loc['2021-12-31':'2022-01-07']

past_cov_train = feature_cols.loc[:'2021-12-31']
past_cov_val = feature_cols.loc['2021-12-31':'2022-01-07']


In [5]:
past_cov_train_series = TimeSeries.from_dataframe(past_cov_train)
y_train_series = TimeSeries.from_dataframe(y_train)
past_cov_val_series = TimeSeries.from_dataframe(past_cov_val)
y_val_series = TimeSeries.from_dataframe(y_val)


In [6]:
lgb_train = lgb.Dataset(past_cov_train_series.pd_dataframe(), label=y_train_series.pd_dataframe().values)
lgb_val = lgb.Dataset(past_cov_val_series.pd_dataframe(), label=y_val_series.pd_dataframe().values, reference=lgb_train)


In [11]:
lgb_train.__dict__


{'_handle': c_void_p(6046231168),
 'data': component  temperature_2m (°C)  relative_humidity_2m (%)  \
 time                                                       
 0                         -1.4                      97.0   
 1                         -0.1                      97.0   
 2                         -0.9                      97.0   
 3                         -1.5                      97.0   
 4                         -1.4                      97.0   
 ...                        ...                       ...   
 2017                       6.4                      79.0   
 2018                       5.5                      84.0   
 2019                       5.2                      85.0   
 2020                       5.1                      87.0   
 2021                       4.9                      89.0   
 
 component  apparent_temperature (°C)  precipitation (mm)  rain (mm)  \
 time                                                                  
 0                 

In [7]:
params = {
    'objective': 'regression',
    'metric': 'rmse',
    'boosting_type': 'gbdt',
    'num_leaves': 31,
    'learning_rate': 0.01,
    'feature_fraction': 0.9,
}
num_round = 1000


In [8]:
tscv = TimeSeriesSplit(n_splits=5)
cv_rmse = []


In [9]:
model = lgb.train(params,
                  train_set = lgb_train,
                  num_boost_round = num_round,
                  valid_sets=[lgb_train, lgb_val])


TypeError: Wrong type(ndarray) for label.
It should be list, numpy 1-D array or pandas Series

In [ ]:
def eval_global_model(train_series, val_series, model):
    preds = model.predict(past_cov_val_series.pd_dataframe())

    preds = pd.DataFrame(preds, index=val_series.index, columns=val_series.columns)

    rmse = np.sqrt(mean_squared_error(val_series.pd_dataframe(), preds))

    return rmse


In [ ]:
validation_rmse = eval_global_model(past_cov_train_series, y_val_series, model)
print(f"val RMSE = {validation_rmse}")
